In [0]:
import pyspark.sql.functions as F

#Create a teble in db defult to write the current user
current_user = (spark
                .createDataFrame([("",)], "user string")
                .withColumn("user", F.expr("current_user"))
                .head()["user"]
               )

def get_directory_path_or_folder(get_folder=False):
    directories = dbutils.fs.ls(f"/FileStore/{current_user}")
    directories = spark.createDataFrame(directories)
    directories = directories.filter(F.col("name").startswith("structured")).collect()[0]
    if get_folder != True:
        folder_name = directories[1]
        return(folder_name)
    else:
        structured_path = directories[0]
        return(structured_path)


def get_database():
    database_list = spark.catalog.listDatabases()
    database_rdd = spark.sparkContext.parallelize(database_list)
    database_df = database_rdd.toDF()    
    return(database_df)


def create_database(db_name, db_path) -> bool:
    isCreated_db = False
    if len(db_name) > 0 and len(db_path) > 0:
        spark.sql(f"""
        CREATE DATABASE IF NOT EXISTS {db_name} COMMENT 'structured zone database' LOCATION '{db_path}'
        """)
        isCreated_db = True
    else:
        print('Inser the name and path to database')
        return   
        
    show_database(db_name,isCreated_db)
    
    return(isCreated_db)

def show_database(db_name,isCreated_db: bool):
    if isCreated_db == True and len(db_name) > 0:
        spark.sql(f"""
        DESCRIBE DATABASE EXTENDED {db_name}
        """).display()
        return   

In [0]:
db_name = get_directory_path_or_folder(get_folder=False)[0:10]
db_path = get_directory_path_or_folder(get_folder=True)
create_database(db_name, db_path)
